In [ ]:
!pip install python-binance

In [ ]:
!pip install TA

In [ ]:
from binance import Client
import pandas as pd
import numpy as np
import time
import ta

In [ ]:
#Fill in your Binance API Key inside the ''
api_key = ''
#Fill in your Binance API Secret Key inside the ''
api_secret = ''
client = Client(api_key, api_secret)

In [ ]:
# Pulling Price Data from Binance
def getminutedata(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' min ago UTC'))
  frame = frame.iloc[:,:6]
  frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit = 'ms')
  frame = frame.astype(float)
  return frame

In [ ]:
df = getminutedata('DOTUSDT', '1m', '100')

In [ ]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-04-29 13:04:00,16.47,16.49,16.47,16.49,1184.80
2022-04-29 13:05:00,16.49,16.50,16.48,16.48,2364.43
2022-04-29 13:06:00,16.48,16.48,16.46,16.47,1552.15
2022-04-29 13:07:00,16.47,16.47,16.44,16.45,1690.37
2022-04-29 13:08:00,16.45,16.46,16.42,16.45,2159.22
...,...,...,...,...,...
2022-04-29 14:39:00,16.55,16.55,16.54,16.55,1516.37
2022-04-29 14:40:00,16.55,16.56,16.55,16.56,1216.36
2022-04-29 14:41:00,16.56,16.56,16.54,16.54,1140.58


In [ ]:
def applytechnicals(df):
  df['%K'] = ta.momentum.stoch(df.High,df.Low,df.Close, window = 14, smooth_window = 3)
  df['%D'] = df['%K'].rolling(3).mean()
  df['rsi'] = ta.momentum.rsi(df.Close, window = 14)
  df['macd'] = ta.trend.macd_diff(df.Close)
  df.dropna(inplace=True)

In [ ]:
applytechnicals(df)

In [ ]:
df

,Open,High,Low,Close,Volume,%K,%D,rsi,macd
Time,,,,,,,,,
2022-04-29 13:37:00,16.55,16.56,16.52,16.53,3136.06,57.142857,66.666667,53.392389,0.001161
2022-04-29 13:38:00,16.53,16.54,16.52,16.53,1837.03,57.142857,61.904762,53.392389,-0.000116
2022-04-29 13:39:00,16.54,16.55,16.52,16.52,3143.87,50.000000,54.761905,50.855962,-0.001680
2022-04-29 13:40:00,16.53,16.55,16.53,16.54,4431.70,64.285714,57.142857,55.417602,-0.001424
2022-04-29 13:41:00,16.54,16.56,16.54,16.55,2667.49,71.428571,61.904762,57.539805,-0.000675
...,...,...,...,...,...,...,...,...,...
2022-04-29 14:39:00,16.55,16.55,16.54,16.55,1516.37,100.000000,91.666667,57.219037,0.008345
2022-04-29 14:40:00,16.55,16.56,16.55,16.56,1216.36,100.000000,100.000000,59.414200,0.009150
2022-04-29 14:41:00,16.56,16.56,16.54,16.54,1140.58,84.615385,94.871795,53.501364,0.007961


In [ ]:
class Signals:
  def __init__(self, df, lags):
    self.df = df
    self.lags = lags

  def gettrigger(self):
    dfx = pd.DataFrame()
    for i in range(self.lags + 1):
      mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
      dfx = dfx.append(mask, ignore_index = True)
    return dfx.sum(axis = 0)
  def decide(self):
    self.df['trigger'] = np.where(self.gettrigger(), 1, 0)
    self.df['Buy'] = np.where((self.df.trigger) & (self.df.rsi > 50) & (self.df.macd > 0), 1, 0)


In [ ]:
inst = Signals(df, 5)

In [ ]:
inst.decide()

In [56]:
df

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2022-04-29 13:37:00,16.55,16.56,16.52,16.53,3136.06,57.142857,66.666667,53.392389,0.001161,0,0
2022-04-29 13:38:00,16.53,16.54,16.52,16.53,1837.03,57.142857,61.904762,53.392389,-0.000116,0,0
2022-04-29 13:39:00,16.54,16.55,16.52,16.52,3143.87,50.000000,54.761905,50.855962,-0.001680,0,0
2022-04-29 13:40:00,16.53,16.55,16.53,16.54,4431.70,64.285714,57.142857,55.417602,-0.001424,0,0
2022-04-29 13:41:00,16.54,16.56,16.54,16.55,2667.49,71.428571,61.904762,57.539805,-0.000675,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29 14:39:00,16.55,16.55,16.54,16.55,1516.37,100.000000,91.666667,57.219037,0.008345,0,0
2022-04-29 14:40:00,16.55,16.56,16.55,16.56,1216.36,100.000000,100.000000,59.414200,0.009150,0,0
2022-04-29 14:41:00,16.56,16.56,16.54,16.54,1140.58,84.615385,94.871795,53.501364,0.007961,0,0


In [ ]:
def strategy(pair, qty, open_position=False):
  df = getminutedata(pair, '1m', '100')
  applytechnicals(df)
  inst = Signals(df, 5)
  inst.decide()
  print(f'Current Close is '+str(df.Close.iloc[-1]))
  if df.Buy.iloc[-1]:
    order = client.create_order(symbol = pair, side = 'BUY', type = 'MARKET', quantity = qty)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position = True
  while open_position:
    time.sleep(0.5)
    df = getminutedata(pair, '1m', '2')
    print(f'Current Close'+ str(df.Close.iloc[-1]))
    print(f'Current Target'+ str(df.buyprice * 1.005))
    print(f'Current Stop is'+ str(df.buyprice * 0.995))
    if df.Close[-1] <= buyprice * 0.995 or df.Close[-1] >= 1.005 * buyprice:
      order = client.create_order(symbol = pair, side = 'SELL', type = 'MARKET', quantity = qty)
      print(order)
      break

In [ ]:
while True:
  strategy('DOTUSDT', 1)
  time.sleep(1.0)

Current Close is 16.5
Current Close is 16.5


KeyboardInterrupt: ignored